In [ ]:
from telegram import Bot
from telegram.ext import Updater, MessageHandler, Filters, CallbackContext
from bs4 import BeautifulSoup
import requests
import os
from dotenv import load_dotenv
from telegram.ext import JobQueue, Job
import logging

# Setup logging (optional but useful)
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)

# Load .env variables
load_dotenv()
bot_token = os.getenv('BOT_TOKEN')
chat_id = os.getenv('CHAT_ID')

# Create bot instance
bot = Bot(token=bot_token)
listOfMatches = []
EachCurrentMatch = []

def fetch_matches(context: CallbackContext = None):
    global listOfMatches, EachCurrentMatch

    url = "https://www.cricbuzz.com/"
    headers = {"User-Agent": "Mozilla/5.0"}
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "lxml")

    container = soup.find('div', id="match_menu_container")
    container2 = container.find('div', class_="videos-list-carousal cb-hgt-145")
    EachCurrentMatch = container2.find_all('li', class_="cb-view-all-ga cb-match-card cb-bg-white")

    listOfMatches = []
    msg = "🏏 *Select a match number to get live score:*\n\n"
    for i, match in enumerate(EachCurrentMatch, 1):
        title = match.find('a').get("title")
        listOfMatches.append(title)
        msg += f"{i}. {title}\n"

    # Only send updates if being called by the scheduler
    if context:
        context.bot.send_message(chat_id=chat_id, text=msg, parse_mode="Markdown")

def handle_message(update, context):
    text = update.message.text.strip()

    if text.isdigit():
        matchNo = int(text)
        if 1 <= matchNo <= len(EachCurrentMatch):
            match = EachCurrentMatch[matchNo - 1]
            result = match.find('div', class_="cb-mtch-crd-state cb-ovr-flo cb-font-12 cb-text-complete")
            team1 = match.find('div', class_="cb-hmscg-tm-bat-scr cb-font-14")
            team2 = match.find('div', class_="cb-hmscg-tm-bwl-scr cb-font-14")
            live = match.find('div', class_="cb-mtch-crd-state cb-ovr-flo cb-font-12 cb-text-apple-red")

            if (result or live) and team1 and team2:
                team1_info = team1.find_all('div')
                team2_info = team2.find_all('div')

                if result and len(team1_info) >= 3 and len(team2_info) >= 3:
                    reply = (
                        f"🏏 *{team1_info[0].text}* - {team1_info[2].text}\n"
                        f"🏏 *{team2_info[0].text}* - {team2_info[2].text}\n\n"
                        f"📋 {result.text}"
                    )
                elif live and len(team1_info) >= 3 and len(team2_info) >= 3:
                    reply = (
                        f"🏏 *{team1_info[0].text}* - {team1_info[2].text}\n"
                        f"🏏 *{team2_info[0].text}* - {team2_info[2].text}\n\n"
                        f"📢 *LIVE* - {live.text}"
                    )
                else:
                    reply = "⚠️ Score info not fully available."
            else:
                reply = "⏳ Match not started yet."
        else:
            reply = "❌ Invalid number. Please select a valid match number."
    else:
        reply = "⚠️ Please send a valid number."

    update.message.reply_text(reply, parse_mode="Markdown")

# Set up the updater and dispatcher
updater = Updater(token=bot_token, use_context=True)
dispatcher = updater.dispatcher
job_queue = updater.job_queue

# Add message handler
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_message))

# Schedule the job to run every 60 seconds
job_queue.run_repeating(fetch_matches, interval=60, first=1)

# Start polling
updater.start_polling()
print("🤖 Bot is now listening on Telegram...")
